In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/Othercomputers/Мое устройство Компьютер/Google.Disk/Colab Notebooks/data

/content/drive/Othercomputers/Мое устройство Компьютер/Google.Disk/Colab Notebooks/data


In [4]:
path_to_file = 'evgenyi_onegin.txt'

In [5]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [6]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [7]:
text = text + text

In [8]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [9]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [10]:
len(text_as_int), len(text)

(573968, 573968)

### train and target

In [11]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [13]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [14]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [15]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [17]:
def build_model_LSTM(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [24]:
def build_model_GRU(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                                       
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [39]:
model_LSTM = build_model_LSTM(
                vocab_size=len(vocab),
                embedding_dim=embedding_dim,
                rnn_units=rnn_units,
                batch_size=BATCH_SIZE)

model_GRU = build_model_GRU(
                vocab_size=len(vocab),
                embedding_dim=embedding_dim,
                rnn_units=rnn_units,
                batch_size=BATCH_SIZE)

In [40]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model_LSTM(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [41]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model_GRU(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [28]:
model_LSTM.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (64, None, 128)           16768     
                                                                 
 lstm_4 (LSTM)               (64, None, 1024)          4722688   
                                                                 
 lstm_5 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_6 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_7 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense_2 (Dense)             (64, None, 131)           134275    
                                                                 
Total params: 30,051,843
Trainable params: 30,051,843


In [29]:
model_GRU.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 128)         16768     
                                                                 
 gru_3 (GRU)                 (None, None, 1024)        3545088   
                                                                 
 gru_4 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_5 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_6 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense_3 (Dense)             (None, None, 131)         134275    
                                                                 
Total params: 22,588,931
Trainable params: 22,588,931


In [30]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 131), dtype=float32, numpy=
array([[-1.3629129e-04, -4.7431971e-05, -4.0433940e-04, ...,
         6.2282517e-05,  3.0622212e-04,  2.1896471e-04],
       [-3.7538761e-04, -1.7631231e-04, -1.5345920e-03, ...,
        -3.3989367e-05,  6.1750936e-04,  7.6029456e-04],
       [-2.0577875e-04, -2.7565460e-05, -2.6544961e-03, ...,
         1.6791823e-04,  9.4978901e-04,  1.1444013e-03],
       ...,
       [ 1.9948801e-03,  5.6580710e-04, -4.5413761e-03, ...,
        -2.2248162e-03,  1.2810150e-02,  1.3575548e-03],
       [ 1.9946438e-03,  5.6177157e-04, -4.5373766e-03, ...,
        -2.2213571e-03,  1.2816023e-02,  1.3576783e-03],
       [ 1.8185466e-03,  6.6633476e-04, -4.2639826e-03, ...,
        -2.0401236e-03,  1.2456469e-02,  7.9884159e-04]], dtype=float32)>

In [42]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [43]:
print("Input: \n", repr("".join(idx2char[input_example_batch[10]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '            Между Онегиным и мной,\n                        Чтобы насмешливый читатель\n              '

Next Char Predictions: 
 'эхгGгМ,GКg7МLС8A2uA5ШDьADEюоФ?в)ла-THoсp0Ч7{Ш!5\'зШeЦAНfrд ЛНOЧqз4FУLkOз"рт22wрГ5ЬdЬuIэТgйЛРw7DRv\'жКв'


## Train the model

In [44]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8783


In [45]:
model_LSTM.compile(optimizer='adam', loss=loss)
model_GRU.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [46]:
# Directory where the checkpoints will be saved
checkpoint_dir_LSTM = './training_checkpoints_LSTM'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir_LSTM, "LSTM_{epoch}")

checkpoint_callback_LSTM = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

# Directory where the checkpoints will be saved
checkpoint_dir_GRU = './training_checkpoints_GRU'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir_GRU, "GRU_{epoch}")

checkpoint_callback_GRU = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

### Execute the training

In [47]:
EPOCHS = 30

In [48]:
history_LSTM = model_LSTM.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback_LSTM])

Epoch 1/30
88/88 [==============================] - 34s 329ms/step - loss: 2.2849
Epoch 2/30
88/88 [==============================] - 30s 339ms/step - loss: 1.6960
Epoch 3/30
88/88 [==============================] - 32s 361ms/step - loss: 1.5038
Epoch 4/30
88/88 [==============================] - 31s 347ms/step - loss: 1.4146
Epoch 5/30
88/88 [==============================] - 31s 350ms/step - loss: 1.3854
Epoch 6/30
88/88 [==============================] - 34s 381ms/step - loss: 1.3585
Epoch 7/30
88/88 [==============================] - 33s 365ms/step - loss: 1.3179
Epoch 8/30
88/88 [==============================] - 33s 366ms/step - loss: 1.3002
Epoch 9/30
88/88 [==============================] - 34s 377ms/step - loss: 1.2826
Epoch 10/30
88/88 [==============================] - 33s 367ms/step - loss: 1.2677
Epoch 11/30
88/88 [==============================] - 33s 366ms/step - loss: 1.2385
Epoch 12/30
88/88 [==============================] - 35s 386ms/step - loss: 1.2164
Epoch 13/30
8

In [49]:
history_GRU = model_GRU.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback_GRU])

Epoch 1/30
88/88 [==============================] - 31s 281ms/step - loss: 2.1307
Epoch 2/30
88/88 [==============================] - 26s 283ms/step - loss: 1.4681
Epoch 3/30
88/88 [==============================] - 26s 294ms/step - loss: 1.3128
Epoch 4/30
88/88 [==============================] - 25s 280ms/step - loss: 1.2247
Epoch 5/30
88/88 [==============================] - 25s 279ms/step - loss: 1.1418
Epoch 6/30
88/88 [==============================] - 26s 293ms/step - loss: 1.0469
Epoch 7/30
88/88 [==============================] - 26s 285ms/step - loss: 0.9538
Epoch 8/30
88/88 [==============================] - 26s 284ms/step - loss: 0.8895
Epoch 9/30
88/88 [==============================] - 27s 294ms/step - loss: 0.7737
Epoch 10/30
88/88 [==============================] - 25s 281ms/step - loss: 0.6889
Epoch 11/30
88/88 [==============================] - 25s 277ms/step - loss: 0.5760
Epoch 12/30
88/88 [==============================] - 27s 299ms/step - loss: 0.5143
Epoch 13/30
8

## Generate text

In [50]:
tf.train.latest_checkpoint(checkpoint_dir_GRU)

'./training_checkpoints_GRU/GRU_30'

In [51]:
tf.train.latest_checkpoint(checkpoint_dir_LSTM)

'./training_checkpoints_LSTM/LSTM_30'

In [52]:
model_LSTM = build_model_LSTM(vocab_size, embedding_dim, rnn_units, batch_size=1)
model_LSTM.load_weights(tf.train.latest_checkpoint(checkpoint_dir_LSTM))
model_LSTM.build(tf.TensorShape([1, None]))

In [53]:
model_GRU = build_model_GRU(vocab_size, embedding_dim, rnn_units, batch_size=1)
model_GRU.load_weights(tf.train.latest_checkpoint(checkpoint_dir_GRU))
model_GRU.build(tf.TensorShape([1, None]))

In [55]:
model_LSTM.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (1, None, 128)            16768     
                                                                 
 lstm_12 (LSTM)              (1, None, 1024)           4722688   
                                                                 
 lstm_13 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_14 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_15 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 dense_6 (Dense)             (1, None, 131)            134275    
                                                                 
Total params: 30,051,843
Trainable params: 30,051,843


In [56]:
model_GRU.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 128)         16768     
                                                                 
 gru_11 (GRU)                (None, None, 1024)        3545088   
                                                                 
 gru_12 (GRU)                (None, None, 1024)        6297600   
                                                                 
 gru_13 (GRU)                (None, None, 1024)        6297600   
                                                                 
 gru_14 (GRU)                (None, None, 1024)        6297600   
                                                                 
 dense_7 (Dense)             (None, None, 131)         134275    
                                                                 
Total params: 22,588,931
Trainable params: 22,588,931


In [57]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [58]:
text_ = generate_text(model_LSTM, start_string=u"Мой дядя самых честных ")
print(text_)

Мой дядя самых честных грам,
                        В тоске сердечных исталая;
                        Так неприступны для мужчин,
                        Покойный сон;
                                                                                                                                                                                                                                                                                                                                                                 


In [59]:
len(text_)

523

In [60]:
text_ = generate_text(model_GRU, start_string=u"Мой дядя самых честных ")
print(text_)

Мой дядя самых честных певи                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                


In [61]:
len(text_)

523